In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
from scipy.interpolate import make_interp_spline
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import base64
import io
import dash_bootstrap_components as dbc


C:\Users\alexb\AppData\Local\Temp\ipykernel_11516\3676754022.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\alexb\AppData\Local\Temp\ipykernel_11516\3676754022.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:

file_path = r'C:\Users\alexb\OneDrive\Documents\Python Scripts\data_dashboard_app\combined_df.csv'
combined_df = pd.read_csv(file_path)


# Get unique runner names
runners = combined_df['Name'].unique()

# Create traces for each runner
traces = []
for i, runner in enumerate(runners):
    runner_data = combined_df[combined_df['Name'] == runner]
    color = 'rgb(204, 204, 204)' if runner not in ['Top 25% Avg', 'Top 50% Avg', 'Top 75% Avg', 'Bottom 25% Avg'] else None
    trace = go.Scatter(
        x=runner_data['Lap'],
        y=runner_data['Pace'],
        mode='lines',
        name=runner,
        line=dict(color=color, width=0.25),
        opacity=0.35
    )
    traces.append(trace)


#FOR SPLICES


#VARIABLES
# Calculate the quartiles
quartile_10 = combined_df['Total_Time'].quantile(0.10)
quartile_40 = combined_df['Total_Time'].quantile(0.40)
quartile_70 = combined_df['Total_Time'].quantile(0.70)

# Filter data for each quartile
top_25_data = combined_df[combined_df['Total_Time'] <= quartile_10]
top_50_data = combined_df[(combined_df['Total_Time'] >= quartile_10) & (combined_df['Total_Time'] <= quartile_40)]
top_75_data = combined_df[(combined_df['Total_Time'] > quartile_40) & (combined_df['Total_Time'] <= quartile_70)]
bottom_25_data = combined_df[combined_df['Total_Time'] >= quartile_70]

# Calculate average Pace per lap for each quartile
average_top_25 = top_25_data.groupby('Lap')['Pace'].mean()
average_top_50 = top_50_data.groupby('Lap')['Pace'].mean()
average_top_75 = top_75_data.groupby('Lap')['Pace'].mean()
average_bottom_25 = bottom_25_data.groupby('Lap')['Pace'].mean()

# Define the x values for the spline interpolation
x_new = np.linspace(average_top_25.index.min(), average_top_25.index.max(), 300)

# Create spline objects for each quartile
spline_top_25 = make_interp_spline(average_top_25.index, average_top_25.values, k=3)
spline_top_50 = make_interp_spline(average_top_50.index, average_top_50.values, k=3)
spline_top_75 = make_interp_spline(average_top_75.index, average_top_75.values, k=3)
spline_bottom_25 = make_interp_spline(average_bottom_25.index, average_bottom_25.values, k=3)

# Evaluate the spline functions at the new x values
smoothed_top_25 = spline_top_25(x_new)
smoothed_top_50 = spline_top_50(x_new)
smoothed_top_75 = spline_top_75(x_new)
smoothed_bottom_25 = spline_bottom_25(x_new)


In [4]:
# Create the layout
layout = go.Layout(
    title='Pace per Lap for Each Runner',
    xaxis=dict(title='Lap', ticklen=1),
    yaxis=dict(title='Pace', range=[300, 850]),
    width=600,
    showlegend=False,
    legend=dict(x=0, y=1),
    plot_bgcolor='white',
    hovermode='closest'
)

# Create a list containing the existing traces and additional plots
combined_data = traces + [
    go.Scatter(x=x_new, y=smoothed_top_25, mode='lines', name='Top 25% Avg', line=dict(color='red', width=2.25)),
    go.Scatter(x=x_new, y=smoothed_top_50, mode='lines', name='Top 50% Avg', line=dict(color='green', width=2.25)),
    go.Scatter(x=x_new, y=smoothed_top_75, mode='lines', name='Top 75% Avg', line=dict(color='blue', width=2.25)),
    go.Scatter(x=x_new, y=smoothed_bottom_25, mode='lines', name='Bottom 25% Avg', line=dict(color='orange', width=2.25))
]



# Create the figure
figure = go.Figure(data=combined_data, layout=layout)

figure

In [5]:
import dash
import dash_html_components as html
from dash.dependencies import Input, Output
import plottable

# Sample data for the table
data = [
    {'Name': 'John', 'Age': 25, 'City': 'New York'},
    {'Name': 'Alice', 'Age': 30, 'City': 'San Francisco'},
    {'Name': 'Bob', 'Age': 28, 'City': 'Chicago'},
]

# Create a plottable table
table = plottable.Table(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Plottable Table in Dash App"),
    html.Div([
        # Convert the plottable table to HTML using to_html()
        html.Div(table.to_html(), dangerously_allow_html=True),
    ]),
])

# Run the app
if __name__ == '__main__':
    app.run_server(port=8056, debug=True)


AttributeError: 'builtin_function_or_method' object has no attribute 'name'